In [1]:
def map_to_cha(raw):
    ret = "@UTF8\n@Begin\n@Languages:\teng\n@Participants:\tSPE0 GUIDE Speaker, SPE1 FOLLOWER Speaker\n@ID:	eng|llm_debate|SPE0|||||Speaker|||\n@ID:	eng|llm_debate|SPE1|||||Speaker|||\n"
    for line in raw[3:]:
        ret += "*SPE{}:\t{}.\n".format("0" if line[0] == 'g' else '1', line[2:-4])
    return ret + "@End"

import pathlib

folder_path = pathlib.Path('transcripts')
output_path = pathlib.Path('2-9-2025')
for transcript in folder_path.rglob("*"):
    with transcript.open('r') as f:
        with output_path.joinpath('{}.cha'.format(transcript.stem)).open('w') as o:
            o.write(map_to_cha(f.readlines()))

In [ ]:
import re

def process_cha_file(input_file, output_file, n=1):
    with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
        previous_speaker = None
        previous_utterance = None
        previous_word_count = 0

        for line in infile:
            # Check if the line is a speaker utterance
            match = re.match(r'^\*(\w+):\s*(.+)$', line)
            if match:
                speaker, utterance = match.groups()
                word_count = len(utterance.split())

                # Strip trailing period if it exists
                if utterance.endswith('.'):
                    utterance = utterance[:-1]

                # Check if the current utterance should be merged with the previous one
                if (previous_speaker == speaker and
                    (previous_word_count <= n or word_count <= n)):
                    # Merge with the previous utterance
                    combined_utterance = f"{previous_utterance}, {utterance}"
                    outfile.write(f'*{speaker}:\t{combined_utterance}.\n')  # Add period at the end
                    previous_utterance = None  # Reset to avoid double-combining
                else:
                    # Write the previous utterance if it exists
                    if previous_utterance is not None:
                        outfile.write(f'*{previous_speaker}:\t{previous_utterance}.\n')
                    # Update previous speaker, utterance, and word count
                    previous_speaker = speaker
                    previous_utterance = utterance
                    previous_word_count = word_count
            else:
                # Write the previous utterance if it exists
                if previous_utterance is not None:
                    outfile.write(f'*{previous_speaker}:\t{previous_utterance}.\n')
                    previous_utterance = None
                # Write non-utterance lines directly
                outfile.write(line)
                previous_speaker = None

        # Write the last utterance if it exists
        if previous_utterance is not None:
            outfile.write(f'*{previous_speaker}:\t{previous_utterance}.\n')

import pathlib

# Example usage
folder_path = pathlib.Path('2-9-2025')
output_path = pathlib.Path('test')
i = 0
for transcript in folder_path.rglob("*"):
    if i > 15:
        break
    process_cha_file(transcript, output_path.joinpath('{}.cha'.format(transcript.stem)), n = 10)
    i += 1


In [9]:
import socratic_model

folder = 'test'

analysis = socratic_model.Analysis(folder)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
analysis.evm(20)
analysis.levenshtein(20)
analysis.workbook.close()

EVM:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

Levenshtein:   0%|          | 0/1 [00:00<?, ?it/s]